## Prérequis

Chromium webdriver installation:
https://chromedriver.chromium.org/downloads

BeautifulSoup

Selenium

## Installation

In [1]:
##Installation de Selenium
pip install selenium

SyntaxError: invalid syntax (1798081938.py, line 2)

# Code

In [1]:
##Installation des packages et webdriver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import urllib3
import re
import time
from collections import defaultdict
import numpy as np
import tqdm
import itertools

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

We start by using BS 

In [2]:
##Define Function to get the html code
def get_page(urlpage, element ,html_class):

    # Get page in html
    req = urllib3.PoolManager()
    res = req.request('GET', urlpage)
    soup = BeautifulSoup(res.data, 'html.parser')
    
    # Return elements that matched the html class in a list
    content = soup.find_all(element ,class_= html_class)
    
    return content 

In [3]:
##Define function to get all links for all theaters
def get_all_links():

    # Webpage from which we extract all links
    urlpage = 'https://www.linternaute.com/cinema/seances/ville/35074/strasbourg/'
   
    # Use get_page function to extract a list with all theaters
    contentCine = get_page(urlpage, 'a', 'jCinemaLink')
    liste_cinemas = re.findall('">(.*?)</a>', str(contentCine))
    
    #Sub the string to get working links
    contentCine_sub = re.sub('href="', 'https://www.linternaute.com', str(contentCine))
    liste_Links = re.findall('jCinemaLink" (.*?)/">',str(contentCine_sub))
    
    # Create an empty dict
    names_links = {}
    
    # Iterate over each link
    for (name, link) in zip(liste_cinemas, liste_Links):
            names_links.update({name : link})
            
    return names_links

In [4]:
#Get all links from all theaters
links = get_all_links()

In [5]:
#Create dictionary
cinemas = {} 
i = 1
for key in links.keys(): 
    cinemas[i] = key 
    i = i+ 1

In [6]:
##Theater choices
print("list of theaters:")

for cinema in cinemas:
    print(cinema,":",cinemas[cinema])

print("===========")

    
    
#Type input
x = int(input("Type the number of the cinema you want to visit:"))
cinema_selected = links[cinemas[x]]

list of theaters:
1 : UGC Ciné Cité Strasbourg
2 : Ciné Vox
3 : Star
4 : Star Saint-Exupéry
Type the number of the cinema you want to visit:2


We continue the rest of the code by using Selenium

In [7]:
##Create a function that opens the driver for the selected theater
def init_driver(url):
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(url) 
    time.sleep(5)
    driver.switch_to.frame(driver.find_element(By.XPATH,'//*[@id="appconsent"]/iframe'))
    element = driver.find_element(By.XPATH, "/html/body/div/div/div/div/div/div/div[2]/aside/section/button[2]")
    element.click()
    driver.switch_to.default_content()
    return driver

In [8]:
#Get dates for a theater

dates = {}

driver = init_driver(cinema_selected) 
dates_id = driver.find_elements(By.CLASS_NAME, "jCinemaCalendarLabel ") #ID dates

#Fill dictionary with dates
for date in dates_id:
    dates[date.get_attribute("data-date")] = date.get_attribute('for') 
#close driver
driver.quit() 

In [11]:
#Dictionary with choice options
dates_dict = {} 
i = 1
for key in dates.keys():
    dates_dict[i] = key
    i = i + 1

print("Dates to choose:")
for date in dates_dict:
    print(date,":",dates_dict[date])
print("========")

x = int(input("Select a date:"))
date_selected_id = dates[dates_dict[x]]

Dates to choose:
1 : 2023-03-12
2 : 2023-03-13
3 : 2023-03-14
4 : 2023-03-15
5 : 2023-03-16
6 : 2023-03-17
7 : 2023-03-18
Select a date:2


In [23]:
date_selected_id 

'id-jour-608-13'

In [12]:
##Define a function to get showtimes of all movies for a selected theater and selected date

def get_funciones(cinema_selected,date_id):    

    driver = init_driver(cinema_selected)

#Clicks the chosen date in the website
    element = driver.find_element(By.CSS_SELECTOR, f"[for={date_id}]") 
    print(element.get_attribute("data-date"))
    print(element.get_attribute('for'))
    element.click()
    time.sleep(4)
    
    
#Get all movies in VF
    vf_list = driver.find_elements(By.CSS_SELECTOR, "[class='showtimes'][data-diffusion='En VF']")
    funciones_vf = {}
    for vf in vf_list:
        horas = vf.find_elements(By.CLASS_NAME, "jShowtimeHour")
        horas_text = []
        for i in horas:
            horas_text.append(i.text)
        funciones_vf[vf.get_attribute("data-movieid")] = horas_text

#Get all movies in VO
    vo_list = driver.find_elements(By.CSS_SELECTOR, "[class='showtimes'][data-diffusion='En VO']")
    funciones_vo = {}
    for vo in vo_list:
        horas = vo.find_elements(By.CLASS_NAME, "jShowtimeHour")
        horas_text = []
        for i in horas:
            horas_text.append(i.text)
        funciones_vo[vo.get_attribute("data-movieid")] = horas_text

#Find movies and put them in a dictionary        
    movies = driver.find_elements(By.CSS_SELECTOR, "[class='jMovie movie']")
    funciones_names = {}
    for movie in movies:
            name = movie.find_element(By.CLASS_NAME,"movie_title")
            funciones_names[movie.get_attribute("data-movieid")] = name.text
#Put all dicts together
    funciones = [] 
    for key_id in funciones_names:
        dict_ = {}
        dict_["id_"] = key_id
        dict_["name"] = funciones_names[key_id]
        if key_id in funciones_vf:
            dict_["VF"] = funciones_vf[key_id]
        if key_id in funciones_vo:
            dict_["VO"] = funciones_vo[key_id]
        funciones.append(dict_)
    driver.quit()
    return funciones

In [13]:
test = get_funciones(cinema_selected, date_selected_id) 

2023-03-13
id-jour-425-13


In [14]:
test

[{'id_': '2697069',
  'name': 'Alibi.com 2',
  'VF': ['13:30', '15:55', '17:45', '20:10', '22:05']},
 {'id_': '2685362',
  'name': "Astérix et Obélix : L'Empire du Milieu",
  'VF': ['13:30', '15:45']},
 {'id_': '2677164',
  'name': 'Ant-Man and the Wasp : Quantumania',
  'VF': ['13:30', '20:05'],
  'VO': ['22:30']},
 {'id_': '2703223',
  'name': 'Creed III',
  'VF': ['14:00', '16:50', '19:45', '22:15'],
  'VO': ['15:30', '22:30']},
 {'id_': '2683446',
  'name': 'The Fabelmans',
  'VF': ['13:30'],
  'VO': ['16:30', '19:30']},
 {'id_': '2750765',
  'name': 'Scream VI',
  'VF': ['14:00', '16:45', '19:45'],
  'VO': ['22:30']},
 {'id_': '2662760', 'name': 'Arrête avec tes mensonges', 'VF': ['18:00']},
 {'id_': '2703235',
  'name': 'The Whale',
  'VF': ['13:30', '15:45', '17:45', '22:30'],
  'VO': ['20:05']},
 {'id_': '2750769', 'name': 'Comme une actrice', 'VF': ['18:00', '22:10']},
 {'id_': '2765283', 'name': 'Sage-Homme', 'VF': ['20:00']}]

## THE END

## ANNEXES

Functions that weren't used

In [ ]:
##Define function to get the available dates
def get_all_dates():
    
    urlpage = 'https://www.linternaute.com/cinema/seances/ville/35074/strasbourg/'
    #Sort html code by date
    contentDate = get_page(urlpage, 'label', 'jCinemaCalendarLabel')
    
    #get a list with the available dates
    liste_dates = re.findall('data-date="(.*?)"',str(contentDate))

    #Re-arrange the date format
    liste_dates_clean = []

    for date in liste_dates :
        tmp = date.split('-')
        tmp_clean = '-'.join([tmp[2], tmp[1], tmp[0]])
        liste_dates_clean.append(tmp_clean)
    
    return liste_dates_clean

In [ ]:
dates = get_all_dates()

In [ ]:
def get_Showdays_class() :
    liste_Showdays = []
    
    for date in dates:
        tmp = date.split('-')
        jour = tmp[0]
        Showday = 'jShowDay show-day-'+jour
        liste_Showdays.append(Showday)
    return liste_Showdays